# Trabalhando com Snapshots e Time Travel
### Snapshots (Instantâneos)
O Snapshot é uma "fotografia" imutável do estado do sistema de dados em um ponto específico no tempo.

**O que é:** Uma cópia lógica e instantânea dos dados e metadados. Diferente de um backup tradicional que copia todos os dados, um snapshot frequentemente armazena apenas os ponteiros para os dados originais e registra as alterações subsequentes (técnica copy-on-write).

**Propósito:** Serve como um ponto de restauração rápido e eficiente. Permite que o sistema retorne a um estado conhecido e consistente em caso de erro, corrupção de dados ou falha.

**Característica Chave:** É um registro do estado em um momento exato.

### Time Travel (Viagem no Tempo)
O Time Travel é a capacidade de consultar ou restaurar dados de qualquer um dos Snapshots disponíveis em um período de retenção predefinido.

**O que é:** Um mecanismo que aproveita os Snapshots (ou o histórico de alterações) para permitir que os usuários executem consultas em versões antigas dos dados, como se estivessem "viajando no tempo".

**Propósito:**

**Análise:** Consultar dados históricos para relatórios de fim de período ou auditoria.

**Recuperação Granular:** Restaurar apenas uma tabela, um esquema, ou até mesmo dados excluídos/modificados acidentalmente, sem precisar restaurar o banco de dados inteiro.

**Debugging:** Rastrear a evolução dos dados e entender quando e como uma modificação específica ocorreu.

**Característica Chave:** É a funcionalidade que usa os Snapshots para acessar o histórico de dados.

1. Configuração do Spark

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init('/opt/spark-3.3.0-bin-hadoop3')

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col
from datetime import datetime, timedelta

# Sessão Spark
spark = SparkSession.builder \
    .appName("IcebergWithSpark") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "/home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .getOrCreate()

2. Criando a tabela de vendas usando Iceberg

In [2]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
""")

DataFrame[]

3. Inserir dados usando SQL

In [3]:
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas VALUES
    (1, 'Produto A', 10, 15.5, DATE('2023-11-01')),
    (2, 'Produto B', 5, 22.0, DATE('2023-11-02')),
    (3, 'Produto C', 8, 30.0, DATE('2023-11-03'))
""")

DataFrame[]

4. Visualização dos dados

In [4]:
spark.sql("SELECT * FROM hadoop_catalog.default.vendas").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2023-11-01|
|  2|Produto B|         5| 22.0|2023-11-02|
|  3|Produto C|         8| 30.0|2023-11-03|
+---+---------+----------+-----+----------+



5. Lista de snapshots atuais da tabela vendas

In [5]:
spark.sql("SELECT snapshot_id, committed_at, operation FROM hadoop_catalog.default.vendas.snapshots").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|8769767624485018141|2025-11-03 01:02:28.642|append   |
+-------------------+-----------------------+---------+



6. Incluimos mais dados

In [6]:
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas VALUES
    (4, 'Produto D', 12, 25.0, DATE('2023-11-04')),
    (5, 'Produto E', 7, 18.5, DATE('2023-11-05'))
""")

DataFrame[]

7. Visualização dos dados

In [ ]:
spark.sql("SELECT * FROM hadoop_catalog.default.vendas order by id").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2023-11-01|
|  2|Produto B|         5| 22.0|2023-11-02|
|  3|Produto C|         8| 30.0|2023-11-03|
|  4|Produto D|        12| 25.0|2023-11-04|
|  5|Produto E|         7| 18.5|2023-11-05|
+---+---------+----------+-----+----------+



8. Agora temos dois snapshots

In [7]:
spark.sql("SELECT snapshot_id, committed_at, operation FROM hadoop_catalog.default.vendas.snapshots").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|8769767624485018141|2025-11-03 01:02:28.642|append   |
|7670213411868779066|2025-11-03 01:02:40.339|append   |
+-------------------+-----------------------+---------+



9. Vamos fazer uma atualização nos dados

In [8]:
spark.sql("""
    UPDATE hadoop_catalog.default.vendas 
    SET preco = preco * 1.1 
    WHERE id IN (1, 2)
""")

DataFrame[]

10. Visualizar dados após update

In [9]:
spark.sql("SELECT * FROM hadoop_catalog.default.vendas order by id").show()

+---+---------+----------+------------------+----------+
| id|  produto|quantidade|             preco|data_venda|
+---+---------+----------+------------------+----------+
|  1|Produto A|        10|             17.05|2023-11-01|
|  2|Produto B|         5|24.200000000000003|2023-11-02|
|  3|Produto C|         8|              30.0|2023-11-03|
|  4|Produto D|        12|              25.0|2023-11-04|
|  5|Produto E|         7|              18.5|2023-11-05|
+---+---------+----------+------------------+----------+



11. Ver todos os snapshots

In [10]:
spark.sql("SELECT snapshot_id, committed_at, operation FROM hadoop_catalog.default.vendas.snapshots").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|8769767624485018141|2025-11-03 01:02:28.642|append   |
|7670213411868779066|2025-11-03 01:02:40.339|append   |
|8582967141202443056|2025-11-03 01:02:50.729|overwrite|
+-------------------+-----------------------+---------+



12. Time Travel - consultar dados de um snapshot específico
- Primeiro, vamos pegar o ID do primeiro snapshot


In [11]:
snapshots = spark.sql("SELECT snapshot_id FROM hadoop_catalog.default.vendas.snapshots ORDER BY committed_at")
first_snapshot = snapshots.collect()[0][0]
print(f"Primeiro snapshot ID: {first_snapshot}")


Primeiro snapshot ID: 8769767624485018141


- Consultar dados do primeiro snapshot

In [12]:
spark.sql(f"""
    SELECT * FROM hadoop_catalog.default.vendas 
    VERSION AS OF {first_snapshot}
    ORDER BY id
""").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2023-11-01|
|  2|Produto B|         5| 22.0|2023-11-02|
|  3|Produto C|         8| 30.0|2023-11-03|
+---+---------+----------+-----+----------+



13. Histórico de mudanças na tabela

In [13]:
spark.sql("SELECT * FROM hadoop_catalog.default.vendas.history").show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-11-03 01:02:28.642|8769767624485018141|null               |true               |
|2025-11-03 01:02:40.339|7670213411868779066|8769767624485018141|true               |
|2025-11-03 01:02:50.729|8582967141202443056|7670213411868779066|true               |
+-----------------------+-------------------+-------------------+-------------------+



14. Informações sobre arquivos da tabela

In [14]:
spark.sql("SELECT file_path, file_format, record_count FROM hadoop_catalog.default.vendas.files").show(truncate=False)

+---------------------------------------------------------------------------------------------------------+-----------+------------+
|file_path                                                                                                |file_format|record_count|
+---------------------------------------------------------------------------------------------------------+-----------+------------+
|/home/tavares/warehouse/default/vendas/data/00045-10-5aa7c052-dd2f-44e8-a5a7-5adc69180d7e-0-00001.parquet|PARQUET    |1           |
|/home/tavares/warehouse/default/vendas/data/00049-11-5aa7c052-dd2f-44e8-a5a7-5adc69180d7e-0-00001.parquet|PARQUET    |1           |
|/home/tavares/warehouse/default/vendas/data/00000-5-c50590d3-dfc9-4763-93ba-c3df93ac6b09-0-00001.parquet |PARQUET    |1           |
|/home/tavares/warehouse/default/vendas/data/00001-6-c50590d3-dfc9-4763-93ba-c3df93ac6b09-0-00001.parquet |PARQUET    |1           |
|/home/tavares/warehouse/default/vendas/data/00002-2-f5b4e7ba-fa89-47